In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os
import time

In [2]:
# Get the paths
path_A = '../DATA/imdb3_neg_nan.csv'
path_B = '../DATA/thenumbers3_neg_nan.csv'
path_C = '../DATA/AllTuplePairs.csv'
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
C = em.read_csv_metadata(path_C, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')


No handlers could be found for logger "py_entitymatching.io.parsers"


In [3]:
C.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
0,4146,1757,11,Hamlet,1996,PG-13,242 min,Drama,Kenneth Branagh,"Kenneth Branagh,Julie Christie,Derek Jacobi,Kate Winslet",$4.41M,Hamlet,1996,PG-13,-1,Drama,-1,Kenneth Branagh,"$7,129,670"
1,5635,2188,15,Six Degrees of Separation,1993,R,112 min,"Comedy, Drama, Mystery",Fred Schepisi,"Will Smith,Stockard Channing,Donald Sutherland,Ian McKellen",$6.41M,A Home of Our Own,1993,Not Rated,-1,-1,Tony Bill,-1,-1
2,5646,168,15,In the Name of the Father,1993,R,133 min,"Biography, Drama",Jim Sheridan,"Daniel Day-Lewis,Pete Postlethwaite,Alison Crosbie,Philip King",$25.01M,A Home of Our Own,1993,Not Rated,-1,-1,Tony Bill,-1,-1
3,5738,458,15,Batman: Mask of the Phantasm,1993,PG,76 min,"Animation, Action, Adventure",Eric Radomski,"Bruce Timm,Kevin Altieri,Boyd Kirkland,Frank Paur,Dan Riba,Kevin Conroy,Dana Delany,Hart Bochner...",$5.62M,A Home of Our Own,1993,Not Rated,-1,-1,Tony Bill,-1,-1
4,5838,778,15,The Remains of the Day,1993,PG,134 min,"Drama, Romance",James Ivory,"Anthony Hopkins,Emma Thompson,John Haycraft,Christopher Reeve",$22.95M,A Home of Our Own,1993,Not Rated,-1,-1,Tony Bill,-1,-1


In [4]:
G = em.read_csv_metadata('../DATA/G.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
#Build and train model on G
rf = em.RFMatcher(name='RF', random_state=0)
feature_table = em.get_features_for_matching(A, B,
                             validate_inferred_attr_types=False)
H = em.extract_feature_vecs(G, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)
rf.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')



In [5]:
#convert All Tuple Pairs to features
CF = em.extract_feature_vecs(C, 
                            feature_table=feature_table, 
                            #attrs_after='gold',
                            show_progress=False)


In [6]:
#Save matching and non matching pairs
#apply to C4
predictions = rf.predict(table=CF,#The entire evaluation set
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='predicted', inplace=False)

In [7]:
P = predictions.predicted
CP = C[P==1]
#tit = [C.ltable_title, C.rtable_title]
allData = pd.concat([CP.ltable_id, CP.ltable_title, CP.rtable_id, CP.rtable_title],axis =1)
allData.to_csv('../DATA/MatchPredctionsOnAllTuplePairs.csv', index=False)

In [8]:
allData

,ltable_id,ltable_title,rtable_id,rtable_title
0,1757,Hamlet,11,Hamlet
25,3547,Home for the Holidays,22,Home for the Holidays
40,502,Hot Fuzz,36,Hot Fuzz
49,3688,Drop Dead Gorgeous,57,Drop Dead Gorgeous
73,3517,Bye Bye Birdie,98,Bye Bye Birdie
85,2558,The Butterfly Effect,123,The Butterfly Effect
101,4100,Mystic Pizza,135,Mystic Pizza
102,1135,Mystic River,139,Mystic River
103,114,Butch Cassidy and the Sundance Kid,141,Butch Cassidy and the Sundance Kid
106,1289,Mystery Train,151,Mystery Train
